# Tutorial 5
In this tutorial, we are going to train a small Convolutional Neural Network using TensorFlow and convert it to an SNN using the few-spike encoding scheme.

The ANN and converted SNN both achieve around 99% on the MNIST test set.

## Install
Download wheel file

In [1]:
if "google.colab" in str(get_ipython()):
    # **YUCK** uninstall newer packages with dependencies which clash with TF 2.14
    !pip uninstall -y tensorstore orbax-checkpoint flax grpcio-status thinc tf-keras jax jaxlib tensorflow-text tensorflow-decision-forests ydf keras-hub optax dopamine-rl google-cloud-pubsub tensorflow-hub spacy chex bigframes fastai

    # **YUCK** install slightly older-than-latest version of numpy 1.XX to resolve weird recursion error with ml_types
    !pip install numpy==1.23.5

    !gdown 1QF6eMWoqmOehbzXNSUbrImyBo0dTbv6J
    !pip install pygenn-5.2.0-cp311-cp311-linux_x86_64.whl
    %env CUDA_PATH=/usr/local/cuda
    !rm -rf /content/ml_genn-ml_genn_2_3_1
    !wget https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_3_1.zip
    !unzip -q ml_genn_2_3_1.zip
    !pip install ./ml_genn-ml_genn_2_3_1/ml_genn
    !pip install ./ml_genn-ml_genn_2_3_1/ml_genn_tf

Downloading...
From: https://drive.google.com/uc?id=1QF6eMWoqmOehbzXNSUbrImyBo0dTbv6J
To: /content/pygenn-5.2.0-cp311-cp311-linux_x86_64.whl
100% 8.60M/8.60M [00:00<00:00, 46.2MB/s]
Processing ./pygenn-5.2.0-cp311-cp311-linux_x86_64.whl
pygenn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
env: CUDA_PATH=/usr/local/cuda
--2025-04-26 14:43:38--  https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_3_1.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_3_1 [following]
--2025-04-26 14:43:38--  https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_3_1
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112

## Train ANN
Firstly we define a simple ANN in Keras with two convolutional layers followed by two dense layers and train it:

In [2]:
from tensorflow.keras import models, layers, datasets
from tensorflow.config import experimental

# Irritatingly, TF's default GPU memory allocator  allocates
# all available GPU memory - this can't be freed and would leave
# none for mlGeNN so we turn off this behaviour
for gpu in experimental.list_physical_devices("GPU"):
    experimental.set_memory_growth(gpu, True)

# Load MNIST data and normalise to [0,1]
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()
train_x = train_x.reshape((-1, 28, 28, 1)) / 255.0
test_x = test_x.reshape((-1, 28, 28, 1)) / 255.0

# Create and compile TF model
tf_model = models.Sequential([
    layers.Conv2D(16, 5, padding="valid", activation="relu", use_bias=False, input_shape=train_x.shape[1:]),
    layers.AveragePooling2D(2),
    layers.Conv2D(8, 5, padding="valid", activation="relu", use_bias=False),
    layers.AveragePooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu", use_bias=False),
    layers.Dense(64, activation="relu", use_bias=False),
    layers.Dense(train_y.max() + 1, activation="softmax", use_bias=False),
], name="simple_cnn")
tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit TF model
tf_model.fit(train_x, train_y, epochs=10)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2387 - accuracy: 0.9271
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0815 - accuracy: 0.9753
Epoch 3/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0582 - accuracy: 0.9817
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0454 - accuracy: 0.9857
Epoch 5/10
1875/1875 [==============================] - 22s 11ms/step - loss: 0.0379 - accuracy: 0.9876
Epoch 6/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0319 - accuracy: 0.9905
Epoch 7/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0266 - accuracy: 0.9915
Epoch 8/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0232 - accuracy: 0.9926
Epoch 9/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0207 - accuracy: 0.9934

## Evaluate ANN model
Now we evaluate the ANN on the MNIST test set:

In [3]:
tf_model.evaluate(test_x, test_y)

313/313 [==============================] - 2s 4ms/step - loss: 0.0326 - accuracy: 0.9896


[0.03261890634894371, 0.9896000027656555]

## Build normalization dataset
To correctly configure the conversion algorithm, the range of activations in each layer is required. We determine this from a single, randomly selected batch of training data. Slightly awkwardly, mlGeNN takes these as an iterator so we turn them into a TF dataset:

In [4]:
import numpy as np
from tensorflow.data import Dataset, AUTOTUNE

# ML GeNN norm dataset
norm_i = np.random.choice(train_x.shape[0], 128, replace=False)

norm_ds = Dataset.from_tensor_slices((train_x[norm_i], train_y[norm_i]))
norm_ds = norm_ds.batch(128)
norm_ds = norm_ds.prefetch(AUTOTUNE)

## Convert model
We are going to use the few-spike conversion scheme to convert the ANN to an SNN with $k=8$ timesteps per examples:

Stöckl, Christoph, and Wolfgang Maass. 2021. “Optimized Spiking Neurons Can Classify Images with High Accuracy through Temporal Coding with Two Spikes.” Nature Machine Intelligence 3(3): 230–38 ([doi](http://dx.doi.org/10.1038/s42256-021-00311-4))


In [5]:
from ml_genn_tf.converters import FewSpike

# Build few-spike converter
converter = FewSpike(k=8, norm_data=[norm_ds])

# Convert and compile ML GeNN model
net, net_inputs, net_outputs, tf_layer_pops = converter.convert(tf_model)

## Compilation
In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, we ask the converter to build us a suitable compiler and specify batch size and that we don't want connectvity expanded into sparse connectivity.

In [6]:
compiler = converter.create_compiler(prefer_in_memory_connect=False, batch_size=128)
compiled_net = compiler.compile(net, inputs=net_inputs, outputs=net_outputs)

## Evaluate SNN models
Finally, we evaluate the SNN model on the MNIST test set:

In [7]:
with compiled_net:
    compiled_net.evaluate({net_inputs[0]: test_x},
                          {net_outputs[0]: test_y})

  0%|          | 0/84 [00:00<?, ?it/s]